# Interval computations - Labs

## Prerequisites

In [0]:
!pip install -q pyinterval

In [0]:
from interval import interval
import pandas as pd
import numpy as np
import math
from numpy.linalg import inv

## **Equation**

### Utility

In [0]:
def width(x_i):
    return sum([x.sup - x.inf for x in x_i])

def midpoint(x_i):
    return (x_i[0].sup + x_i[0].inf) / 2

def itos(x_i):
    return f"[{x_i[0].inf:9.7f}, {x_i[0].sup:9.7f}]"

### Bisection method

#### Bisection with console output

In [0]:
def bisection(f, start, end, tol=0.01, iteration=1):
    X = interval[start, end]
    F = f(X)
    # No root if no zero
    if 0 not in F:
        print(iteration, '-', itos(X), itos(F), sep='\t')
        return
    # Found root if satisfies the tolerance
    if width(X) < tol:
        print(iteration, '+', itos(X), itos(F), sep='\t')
        return
    
    print(iteration, '*', itos(X), itos(F), sep='\t')
    mid = midpoint(X)
    bisection(f, start, mid, tol, iteration + 1)
    bisection(f, mid, end, tol, iteration + 1)

#### Bisection with table output

In [0]:
def bisection_tbl(f, start, end, tol=0.01):
    tbl = pd.DataFrame(columns=['Depth', 'Mark', 'Interval', 'Function extension'])

    def bisection(f, start, end, tol=0.01, iteration=1):
        X = interval[start, end]
        F = f(X)
        # No root if no zero
        if 0 not in F:
            tbl.loc[len(tbl)] = [iteration, '-', itos(X), itos(F)]
            return
        # Found root if satisfies the tolerance
        if width(X) < tol:
            tbl.loc[len(tbl)] = [iteration, '+', itos(X), itos(F)]
            return
        
        tbl.loc[len(tbl)] = [iteration, '*', itos(X), itos(F)]

        mid = midpoint(X)
        bisection(f, start, mid, tol, iteration + 1)
        bisection(f, mid, end, tol, iteration + 1)

    bisection(f, start, end, tol)
    return tbl.sort_values(['Depth', 'Interval'])

#### Example

In [0]:
func = lambda x: x**2 - 8*x + 12
bisection_tbl(func, 5.995, 6.05, 10**-2)

,Depth,Mark,Interval,Function extension
0,1,*,"[5.9950000, 6.0500000]","[-0.4599750, 0.6425000]"
1,2,*,"[5.9950000, 6.0225000]","[-0.2399750, 0.3105063]"
8,2,*,"[6.0225000, 6.0500000]","[-0.1294938, 0.4225000]"
2,3,*,"[5.9950000, 6.0087500]","[-0.1299750, 0.1450766]"
5,3,*,"[6.0087500, 6.0225000]","[-0.0749234, 0.2005063]"
9,3,*,"[6.0225000, 6.0362500]","[-0.0194938, 0.2563141]"
12,3,-,"[6.0362500, 6.0500000]","[0.0363141, 0.3125000]"
3,4,+,"[5.9950000, 6.0018750]","[-0.0749750, 0.0625035]"
4,4,+,"[6.0018750, 6.0087500]","[-0.0474965, 0.0900766]"
6,4,+,"[6.0087500, 6.0156250]","[-0.0199234, 0.1177441]"


### Moore's method

#### Description

$f(x) = 0$, interval $[a, b]$

$f'(x) \in C^1[a,b]$

$F'(x)$ - inclusion monotonic interval extension of $f'(x)$

$N(X) = m(X) + (- \frac{1}{F'(X)}) f(m(X))$

Necessary conditions:  
$f(x)$ has 1 (non-multiple) root at most on $[a, b]$

#### Moore's with console output

In [0]:
def moore(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, '-', itos(X), X_width, sep='\t')
        return
    
    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, '+', itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
        print(iteration, '**', itos(X), X_width, sep='\t')
        moore(f, df, start, x_mid, tol, iteration + 1)
        moore(f, df, x_mid,   end, tol, iteration + 1)
        return
    
    # [Step 3]: Stop if X_next is empty
    f_mid = f(x_mid)
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, '-', itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed interval
    print(iteration, '*', itos(X), X_width, sep='\t')
    moore(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### Moore's with table output

In [0]:
def moore_tbl(f, df, start, end, tol=1e-6, iteration=1):
    tbl = 

    def moore(start, end, iteration=1):
        X = interval([start, end])
        X_width = width(X)

        # [Step 1]: Stop if no 0 in F(X)
        if 0 not in f(X):
            print(iteration, '-', itos(X), X_width, sep='\t')
            return
        
        # Return result if tolerance is satisfied
        if X_width < tol:
            print(iteration, '+', itos(X), X_width, sep='\t')
            return
        
        # [Step 2]: Half if 0 in F'(X)
        x_mid = midpoint(X)
        df_X = df(X)
        if 0 in df_X:
            print(iteration, '**', itos(X), X_width, sep='\t')
            moore(start, x_mid, iteration + 1)
            moore(x_mid,   end, iteration + 1)
            return
        
        # [Step 3]: Stop if X_next is empty
        f_mid = f(x_mid)
        U = x_mid - f_mid / df_X
        X_next = U & X
        if not X_next:
            print(iteration, '-', itos(U) + '&' + itos(X), sep='\t')
            return

        # [Step 4]: Continue with narrowed interval
        print(iteration, '*', itos(X), X_width, sep='\t')
        moore(X_next[0].inf, X_next[0].sup, iteration + 1)

    moore(start, end)
    return tbl.()

#### Example

In [0]:
func = lambda x: x**2 - 8*x + 12
dfunc = lambda x: 2*x - 8

moore(func, dfunc, 3, 7)

1	**	[3.0000000, 7.0000000]	4.0
2	**	[3.0000000, 5.0000000]	2.0
3	**	[3.0000000, 4.0000000]	1.0
4	-	[-0.1875000, 1.5312500]&[3.0000000, 3.5000000]
4	**	[3.5000000, 4.0000000]	0.5
5	-	[3.5000000, 3.7500000]	0.25
5	-	[3.7500000, 4.0000000]	0.25
3	**	[4.0000000, 5.0000000]	1.0
4	**	[4.0000000, 4.5000000]	0.5
5	-	[4.0000000, 4.2500000]	0.25
5	-	[4.2500000, 4.5000000]	0.25
4	-	[6.4687500, 8.1875000]&[4.5000000, 5.0000000]
2	*	[5.0000000, 7.0000000]	2.0
3	+	[6.0000000, 6.0000000]	0.0


### Hansen's method a.k.a. HanSolo's method

#### Hansen's with console output

In [0]:
def hansen(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, '-', itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    f_mid = f(x_mid)
    df_X = df(X)
    if f_mid == 0.0:
        print(iteration, "+", itos(X), X_width, '<====== { f == 0 }', sep='\t')
        hansen(f, df, start, x_mid, tol, iteration + 1)
        hansen(f, df, x_mid, end, tol, iteration + 1)
        return
    else:
        U = x_mid - f_mid / df_X

    # [Step 3]: Stop if X_next is empty
    X_next = U & X
    if not X_next:
        print(iteration, '-', itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed intervals
    print(iteration, "*", itos(X), X_width, sep='\t')
    [hansen(f, df, x.inf, x.sup, tol, iteration + 1) for x in X_next]

#### Hansen's with table output

In [0]:
def hansen_tbl(f, df, start, end, tol=1e-6, iteration=1):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'Interval', 'Width', 'Function extension'])

    def tbl_put(iteration, status, X, X_width, f_X):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), X_width, itos(f_X)]

    def hansen(start, end, iteration=1):
        X = interval([start, end])
        X_width = width(X)
        f_X = f(X)

        # [Step 1]: Stop if no 0 in F(X)
        if 0 not in f_X:
            tbl_put(iteration, 'no root', X, X_width, f_X)
            return

        # Return result if tolerance is satisfied
        if X_width < tol:
            tbl_put(iteration, 'RESULT', X, X_width, f_X)
            return
        
        # [Step 2]: Half if 0 in F'(X)
        x_mid = midpoint(X)
        f_mid = f(x_mid)
        df_X = df(X)
        if f_mid == 0.0:
            tbl_put(iteration, 'RESULT and split', X, X_width, f_X)
            hansen(start, x_mid, iteration + 1)
            hansen(x_mid, end, iteration + 1)
            return
        else:
            U = x_mid - f_mid / df_X

        # [Step 3]: Stop if X_next is empty
        X_next = U & X
        if not X_next:
            tbl_put(iteration, 'empty intersection', X, X_width, f_X)
            return

        # [Step 4]: Continue with narrowed intervals
        tbl_put(iteration, 'no root', X, X_width, f_X)
        [hansen(x.inf, x.sup, iteration + 1) for x in X_next]
    
    hansen(start, end)
    return tbl.sort_values(['Depth', 'Status', 'Interval'])

#### Example

In [0]:
func = lambda x: x**2 - 8*x + 12
dfunc = lambda x: 2*x - 8

hansen_tbl(func, dfunc, -10, 10)

,Depth,Status,Interval,Width,Function extension
0,1,no root,"[-10.0000000, 10.0000000]",2.000000e+01,"[-68.0000000, 192.0000000]"
1,2,no root,"[-10.0000000, -1.0000000]",9.000000e+00,"[21.0000000, 192.0000000]"
2,2,no root,"[0.4285714, 10.0000000]",9.571429e+00,"[-67.8163265, 108.5714286]"
3,3,no root,"[0.4285714, 4.8607143]",4.432143e+00,"[-26.7020408, 32.1979719]"
9,3,no root,"[5.4247449, 10.0000000]",4.575255e+00,"[-38.5721428, 68.6020408]"
8,4,empty intersection,"[3.9011615, 4.8607143]",9.595528e-01,"[-11.6666536, 4.4172517]"
4,4,no root,"[0.4285714, 2.3418219]",1.913250e+00,"[-6.5509015, 14.0555583]"
10,4,no root,"[5.4247449, 6.8972300]",1.472485e+00,"[-13.7499829, 16.1738227]"
5,5,no root,"[1.7824042, 2.2407129]",4.583087e-01,"[-2.7487383, 2.7615608]"
11,5,no root,"[5.9259044, 6.0453828]",1.194784e-01,"[-1.2467192, 1.1394176]"


### [Krawczyk] Kravchyk's method, a.k.a. Kravchenko, a.k.a Kravchuk The President

#### Kravchuk consolnyi

In [0]:
def kravchyk(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, "-", itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "+", itos(X), X_width, sep='\t')
        return

    # Split interval if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
      print(iteration, "**", itos(X), X_width, sep='\t')
      kravchyk(f, df, start, x_mid, tol, iteration + 1)
      kravchyk(f, df, x_mid,   end, tol, iteration + 1)
      return

    # Stop if X_next is empty
    df_x = df(x_mid)
    K = x_mid - f(x_mid) / df_x + (1 - df(X) / df_x)*(X - x_mid)
    X_next = K & X
    if not X_next:
        print(iteration, "-", itos(K) + '&' + itos(X), sep='\t')
        return

    # Display interval if shrunk or not
    print(iteration, "*", itos(X), X_width, sep='\t')
    kravchyk(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### Kravchuk tablichniy

#### Example

In [0]:
func = lambda x: x**3 - 8*x**2 + 17*x - 10
dfunc = lambda x: 3*x**2 - 16*x + 7

kravchyk(func, dfunc, -2, 5)

1	**	[-2.0000000, 5.0000000]	7.0
2	**	[-2.0000000, 1.5000000]	3.5
3	*	[-2.0000000, -0.2500000]	1.75
4	*	[-1.7905640, -0.2500000]	1.5405639571518592
5	*	[-1.6460071, -0.3543975]	1.2916096367255707
6	*	[-1.4537235, -0.5462765]	0.9074470196286358
7	*	[-1.2186022, -0.7813978]	0.4372043494163699
8	*	[-1.0492114, -0.9507886]	0.0984228186526569
9	*	[-1.0024380, -0.9975620]	0.00487602880508442
10	*	[-1.0000059, -0.9999941]	1.1891780640094218e-05
11	+	[-1.0000000, -1.0000000]	7.070743990311712e-11
3	**	[-0.2500000, 1.5000000]	1.75
4	-	[-0.2500000, 0.6250000]	0.875
4	*	[0.6250000, 1.5000000]	0.875
5	-	[1.3283723, 1.5000000]	0.17162771439087243
2	**	[1.5000000, 5.0000000]	3.5
3	**	[1.5000000, 3.2500000]	1.75
4	*	[1.5000000, 2.3750000]	0.875
5	*	[1.5000000, 1.9729434]	0.4729434072345391
6	*	[1.5335052, 1.7262901]	0.19278492303405637
7	*	[1.6116880, 1.6437513]	0.032063224227062204
8	*	[1.6272792, 1.6281582]	0.0008790440233732433
9	+	[1.6277183, 1.6277190]	6.595229367079014e-07
4	-	[1.3269591, 2.043

## **System of equations**

### Moore's method

$N(\vec{X}) = m(\vec{X}) + (- \frac{1}{F'(\vec{X})}) f(m(\vec{X}))$

#### Utility

In [0]:
def print_interval(iteration, status, X, Y):
    print(iteration, status, 'X:'+itos(X), width(X), 'Y:'+itos(Y), width(Y), sep='\t')

# returns determinant, 
def det2x2(m):
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

# subinterval, t - top, b - bottom, l - left, r - right
def half_interval(x):
    mid = midpoint(x)
    return (interval[x[0].inf, mid], interval[mid, x[0].sup])

#### Moore's with console output

In [0]:
# Example:
# f = lambda x, y: (x + y, x - y)
# df = lambda x, y: ((1,  1),
#                    (1, -1))
# intervals = (interval[-1, 1], interval[-1, 1])
#
def moore_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)

    # [Step 1]: Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, '-', X, Y)
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, '+', X, Y)
        return

    # [Step 2]: Half if 0 in F'(X, Y)
    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))

    if 0 in df_XY_det:
        print_interval(iteration, '**', X, Y)
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        if X_width < tol: # half only Y
            moore_system(f, df, (X, Yl), tol, iteration + 1)
            moore_system(f, df, (X, Yr), tol, iteration + 1)
        elif Y_width < tol: # half only X
            moore_system(f, df, (Xl, Y), tol, iteration + 1)
            moore_system(f, df, (Xr, Y), tol, iteration + 1)
        else: # half everything
            moore_system(f, df, (Xl, Yl), tol, iteration + 1)
            moore_system(f, df, (Xl, Yr), tol, iteration + 1)
            moore_system(f, df, (Xr, Yl), tol, iteration + 1)
            moore_system(f, df, (Xr, Yr), tol, iteration + 1)
        return

    # [Step 3]: Stop if X_next or Y_next is empty
    f1m, f2m = f(x_mid, y_mid)
    df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
    det = df_XY_det

    U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
    U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )

    X_next, Y_next = U_X & X, U_Y & Y
    if not X_next or not Y_next:
        print('empty ==> ', end='')
        print_interval(iteration, '-', X, Y)
        return

    # [Step 4]: Continue with narrowed interval
    print_interval(iteration, "*", X, Y)
    moore_system(f, df, (X_next, Y_next), tol, iteration + 1)

#### Moore's with table output

In [0]:
def moore_system_tbl(f, df, intervals, tol=1e-6):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'X', 'X width', 'Y', 'Y width'])

    def tbl_put(iteration, status, X, Y, X_width, Y_width):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), itos(Y), X_width, Y_width]

    def moore_system(intervals, iteration=1):
        X, Y = intervals
        X_width, Y_width = width(X), width(Y)

        # [Step 1]: Stop if no 0 in F(X, Y)
        f_XY = f(X, Y)
        if 0 not in f_XY[0] or 0 not in f_XY[1]:
            tbl_put(iteration, 'no root', X, Y, X_width, Y_width)
            return

        # Return result if tolerance is satisfied
        if max(X_width, Y_width) < tol:
            tbl_put(iteration, 'RESULT', X, Y, X_width, Y_width)
            return

        # [Step 2]: Half if 0 in F'(X, Y)
        x_mid, y_mid = midpoint(X), midpoint(Y)
        df_XY = df(X, Y)
        df_XY_det = interval(det2x2(df_XY))

        if 0 in df_XY_det:
            Xl, Xr = half_interval(X)
            Yl, Yr = half_interval(Y)
            if X_width < tol: # half only Y
                tbl_put(iteration, 'half Y', X, Y, X_width, Y_width)
                moore_system((X, Yl), iteration + 1)
                moore_system((X, Yr), iteration + 1)
            elif Y_width < tol: # half only X
                tbl_put(iteration, 'half X', X, Y, X_width, Y_width)
                moore_system((Xl, Y), iteration + 1)
                moore_system((Xr, Y), iteration + 1)
            else: # half everything
                tbl_put(iteration, 'half X and Y', X, Y, X_width, Y_width)
                moore_system((Xl, Yl), iteration + 1)
                moore_system((Xl, Yr), iteration + 1)
                moore_system((Xr, Yl), iteration + 1)
                moore_system((Xr, Yr), iteration + 1)
            return

        # [Step 3]: Stop if X_next or Y_next is empty
        f1m, f2m = f(x_mid, y_mid)
        df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
        det = df_XY_det

        U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
        U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )

        X_next, Y_next = U_X & X, U_Y & Y
        if not X_next or not Y_next:
            tbl_put(iteration, 'empty intersection', X, Y, X_width, Y_width)
            return

        # [Step 4]: Continue with narrowed interval
        tbl_put(iteration, 'continue', X, Y, X_width, Y_width)
        moore_system((X_next, Y_next), iteration + 1)

    moore_system(intervals)
    return tbl.sort_values(['Depth', 'Status', 'X', 'Y'])

#### Example

In [0]:
func = lambda x, y: (x + y, x - y)
dfunc = lambda x, y: ((1,  1), (1, -1))
init_intervals = (interval[-40, 1], interval[-1, 100])
moore_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-40.0000000, 1.0000000]","[-1.0000000, 100.0000000]",41.0,101.0
1,2,RESULT,"[-0.0000000, 0.0000000]","[-0.0000000, 0.0000000]",0.0,0.0


In [0]:
func = lambda x, y: (x**2 - y - 3,
                     x    - y - 1)
dfunc = lambda x, y: ((2*x, -1),
                      (1,   -1))
init_intervals = (interval[-4, 3.5], interval[-3.7, 5.4])
moore_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,half X and Y,"[-4.0000000, 3.5000000]","[-3.7000000, 5.4000000]",7.500000e+00,9.100000e+00
1,2,continue,"[-4.0000000, -0.2500000]","[-3.7000000, 0.8500000]",3.750000e+00,4.550000e+00
8,2,half X and Y,"[-0.2500000, 3.5000000]","[-3.7000000, 0.8500000]",3.750000e+00,4.550000e+00
20,2,half X and Y,"[-0.2500000, 3.5000000]","[0.8500000, 5.4000000]",3.750000e+00,4.550000e+00
7,2,no root,"[-4.0000000, -0.2500000]","[0.8500000, 5.4000000]",3.750000e+00,4.550000e+00
2,3,continue,"[-1.6093750, -0.2500000]","[-3.7000000, 0.4409722]",1.359375e+00,4.140972e+00
16,3,continue,"[1.6250000, 3.5000000]","[-1.4250000, 0.8500000]",1.875000e+00,2.275000e+00
23,3,continue,"[1.6250000, 3.5000000]","[0.8500000, 3.1250000]",1.875000e+00,2.275000e+00
10,3,half X and Y,"[-0.2500000, 1.6250000]","[-1.4250000, 0.8500000]",1.875000e+00,2.275000e+00
9,3,no root,"[-0.2500000, 1.6250000]","[-3.7000000, -1.4250000]",1.875000e+00,2.275000e+00


In [0]:
func = lambda x, y: (x**3 - x**2 + 3 - y,
                     x**2 + 1 - y)
dfunc = lambda x, y: ((3*x**2 - 2*x, -1),
                      (         2*x, -1))
init_intervals = (interval[-0.9, -0.8], interval[1.5, 1.8])
moore_system_tbl(func, dfunc, init_intervals, tol=1e-6)

,Depth,Status,X,X width,Y,Y width
0,1,continue,"[-0.9000000, -0.8000000]","[1.5000000, 1.8000000]",1.000000e-01,3.000000e-01
1,2,continue,"[-0.8405891, -0.8380579]","[1.6958706, 1.7145996]",2.531167e-03,1.872896e-02
2,3,continue,"[-0.8392874, -0.8392861]","[1.7043988, 1.7044057]",1.332322e-06,6.892175e-06
3,4,RESULT,"[-0.8392868, -0.8392868]","[1.7044023, 1.7044023]",4.551914e-15,3.042011e-14


### Hansen's method a.k.a. HanSolo's method

In [0]:
def hansen(f_1, df_1, f_2, df_2, X_start, X_end, Y_start, Y_end, tol=1e-6, iteration=1):
    X = interval([X_start, X_end])
    Y = interval([Y_start, Y_end])

    X_width = width(X)
    Y_width = width(Y)

    x_mid = midpoint(X)
    y_mid = midpoint(Y)
    

#### Example

### Kravchyk's method, a.k.a. Kravchenko, a.k.a Kravchuk The President

#### Kravchyk with table output

In [0]:
def kravchyk_system_tbl(f, df, intervals, tol=1e-6):
    tbl = pd.DataFrame(columns=['Depth', 'Status', 'X', 'X width', 'Y', 'Y width'])

    def tbl_put(iteration, status, X, Y, X_width, Y_width):
        tbl.loc[len(tbl)] = [iteration, status, itos(X), itos(Y), X_width, Y_width]

    def kravchyk_system(intervals, iteration=1):
        X, Y = intervals
        X_width, Y_width = width(X), width(Y)

        # [Step 1]: Stop if no 0 in F(X, Y)
        f_XY = f(X, Y)
        if 0 not in f_XY[0] or 0 not in f_XY[1]:
            tbl_put(iteration, 'no root', X, Y, X_width, Y_width)
            return

        # Return result if tolerance is satisfied
        if max(X_width, Y_width) < tol:
            tbl_put(iteration, 'RESULT', X, Y, X_width, Y_width)
            return

        # [Step 2]: Half if 0 in F'(X, Y)
        x_mid, y_mid = midpoint(X), midpoint(Y)
        df_XY = df(X, Y)
        df_mid = df(x_mid, y_mid)
        df_XY_det = interval(det2x2(df_XY))
        df_mid_det = interval(det2x2(df_mid))

        if 0 in df_XY_det:
            Xl, Xr = half_interval(X)
            Yl, Yr = half_interval(Y)
            if X_width < tol: # half only Y
                tbl_put(iteration, 'half Y', X, Y, X_width, Y_width)
                kravchyk_system((X, Yl), iteration + 1)
                kravchyk_system((X, Yr), iteration + 1)
            elif Y_width < tol: # half only X
                tbl_put(iteration, 'half X', X, Y, X_width, Y_width)
                kravchyk_system((Xl, Y), iteration + 1)
                kravchyk_system((Xr, Y), iteration + 1)
            else: # half everything
                tbl_put(iteration, 'half X and Y', X, Y, X_width, Y_width)
                kravchyk_system((Xl, Yl), iteration + 1)
                kravchyk_system((Xl, Yr), iteration + 1)
                kravchyk_system((Xr, Yl), iteration + 1)
                kravchyk_system((Xr, Yr), iteration + 1)
            return

        # [Step 3]: Stop if X_next or Y_next is empty
        f1m, f2m = f(x_mid, y_mid)
        df1X, df1Y, df2X, df2Y = *df_XY[0], *df_XY[1]
        df1x, df1y, df2x, df2y = *df_mid[0], *df_mid[1]
        det = df_mid_det

        U_X = x_mid + 
        U_Y = y_mid + ( ( df2X/det)*f1m + (-df1X/det)*f2m )

        X_next, Y_next = U_X & X, U_Y & Y
        if not X_next or not Y_next:
            tbl_put(iteration, 'empty intersection', X, Y, X_width, Y_width)
            return

        # [Step 4]: Continue with narrowed interval
        tbl_put(iteration, 'continue', X, Y, X_width, Y_width)
        moore_system((X_next, Y_next), iteration + 1)

    moore_system(intervals)
    return tbl.sort_values(['Depth', 'Status', 'X', 'Y'])

#### Example

### Old but gold

In [0]:
#@title Interval Matrix

class IntervalMatrix(object):
  @staticmethod
  def from_lists(arrayOfArrays):
    res = IntervalMatrix(len(arrayOfArrays), len(arrayOfArrays[0]))
    res.data = [[interval[bounds[0], bounds[1]] if isinstance(bounds, list) else interval[bounds] for bounds in row] for row in arrayOfArrays]
    return res

  @staticmethod
  def from_intervals(arrayOfArraysOfIntervals):
    res = IntervalMatrix(len(arrayOfArraysOfIntervals), len(arrayOfArraysOfIntervals[0]))
    res.data = [[i for i in row] for row in arrayOfArraysOfIntervals]
    return res
  
  def to_numpy(self):
    return np.array([[val[0].inf, val[0].sup] for row in self.data for val in row])
  
  def __init__(self, rows, columns):
    self.rows = rows
    self.columns = columns

  def __add__(self, other):
    res = IntervalMatrix(self.rows, self.columns)
    res.data = [[val1 + val2 for val1, val2 in zip(row1, row2)] for row1, row2 in zip(self.data, other.data)]
    return res

  def __sub__(self, other):
    res = IntervalMatrix(self.rows, self.columns)
    res.data = [[val1 - val2 for val1, val2 in zip(row1, row2)] for row1, row2 in zip(self.data, other.data)]
    return res

  def __mul__(self, other):
    if not self.columns == other.rows:
      raise "Nigga you blind or somethin?"
    res = IntervalMatrix(self.rows, other.columns)
    res.data = [[sum([v1 * v2 for v1,v2 in zip(leftRow, rightColumn)]) for rightColumn in other.transpose().data] for leftRow in self.data]
    return res

  def transpose(self):
    res = IntervalMatrix(self.columns, self.rows)
    res.data = [[self.data[j][i] for j in range(self.rows)] for i in range(self.columns)]
    return res

  def __getitem__(self, pos):
    row, column = pos
    return self.data[row][column]

  def __setitem__(self, pos, value):
    row, column = pos
    self.data[row][column] = value

  def __str__(self):
    res = ""
    for row in self.data:
      res += "|\t"
      for val in row:
        res += f"[{val[0].inf}; {val[0].sup}]"
      res += "\t|\n"
    return res

  def flatten(self):
    return [item for row in self.data for item in row]

  def inverse(self):
    if self.rows != self.columns:
      raise "Nigga you dumb or somethin"
    det = self[0,0]*self[1,1] - self[0,1]*self[1,0]
    res = IntervalMatrix(2,2)
    res.data[0,0] = self.data[1,1]/det
    res.data[1,1] = self.data[0,0]/det
    res.data[0,1] = -self.data[0,1]/det
    res.data[1,0] = -self.data[1,0]/det
    return res

In [0]:
def flatten(array):
  return [item for row in array for item in row]

In [0]:
def kravchyk_system(f, df, intervals, tol=1e-6, iteration=1):
    X = interval[intervals[0][0][0], intervals[0][0][1]]
    Y = interval[intervals[1][0][0], intervals[1][0][1]]

    X_width = width(X)
    Y_width = width(Y)

    F_XY = f(X, Y)
    # [Step 1]: Stop if no 0 in F(X)
    if all([0 not in f[0] for f in F_XY]):
        print(iteration, "-", 
              "X", itos(X), X_width, 
              "Y", itos(Y), Y_width, 
              sep='\t')
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print(iteration, "+", 
              "X", itos(X), X_width, 
              "Y", itos(Y), Y_width, 
              sep='\t')
        return

    x_mid = midpoint(X)
    y_mid = midpoint(Y)
    dF_XY = df(X, Y)
    
    df_xy = df(x_mid, y_mid)
    df_xy_det = df_xy[0][0]*df_xy[1][1] - df_xy[0][1]*df_xy[1][0]

    # Split interval if 0 in F'(X) or if det F'(X) = 0
    if all([0 in dF for dF in flatten(dF_XY)]) or df_xy_det == 0:
      print(iteration, "**", 
              "X", itos(X), X_width, 
              "Y", itos(Y), Y_width, 
              sep='\t')
      if X_width > Y_width:
        kravchyk_system(f, df, [[X[0].inf, x_mid], intervals[1][0]], tol, iteration + 1)
        kravchyk_system(f, df, [[x_mid, X[0].sup], intervals[1][0]], tol, iteration + 1)
      else:
        kravchyk_system(f, df, [intervals[0][0], [Y[0].inf, y_mid]], tol, iteration + 1)
        kravchyk_system(f, df, [intervals[0][0], [y_mid, Y[0].sup]], tol, iteration + 1)
      return
    e = IntervalMatrix.from_lists([[1, 0], [0, 1]])
    df_xy_inv = IntervalMatrix.from_lists(np.array([[df_xy[1][1], -df_xy[0][1]],
                          [-df_xy[1][0], df_xy[0][0]]]) / df_xy_det)
    xy_mid = IntervalMatrix.from_lists([[x_mid],[y_mid]])
    f_mid = IntervalMatrix.from_intervals(f(x_mid, y_mid))
    dF_XY = IntervalMatrix.from_intervals(dF_XY)
    XY = IntervalMatrix.from_lists(intervals)
    intervalize = np.vectorize(lambda x: interval[x], otypes=[object])

    K = xy_mid - df_xy_inv*f_mid + (e - df_xy_inv*dF_XY)*(XY - xy_mid)
    X_next = K[0,0] & X
    Y_next = K[1,0] & Y

    if not X_next:
        print(iteration, "-", "X", itos(K[0][0]) + '&' + itos(X), sep='\t')
        return
    if not Y_next:
        print(iteration, "-", "Y", itos(K[1][0]) + '&' + itos(Y), sep='\t')
        return
    
    # Display if shrunk or not
    print(iteration, "*", 
          "X", itos(X), X_width, 
          "Y", itos(Y), Y_width, 
          sep='\t')
    
    # Force a split by the widest axis
    if X_next == X or Y_next == Y:
      if X_width > Y_width:
        kravchyk_system(f, df, [[X[0].inf, x_mid], intervals[1][0]], tol, iteration + 1)
        kravchyk_system(f, df, [[x_mid, X[0].sup], intervals[1][0]], tol, iteration + 1)
      else:
        kravchyk_system(f, df, [intervals[0][0], [Y[0].inf, y_mid]], tol, iteration + 1)
        kravchyk_system(f, df, [intervals[0][0], [y_mid, Y[0].sup]], tol, iteration + 1)
      return

    kravchyk_system(f, df, [[[X_next[0].inf, X_next[0].sup]], [[Y_next[0].inf, Y_next[0].sup]]], tol, iteration + 1)

#### Example

In [0]:
f1 = lambda x, y: x**2 + x - 3*y + 10
f2 = lambda x, y: x**3 + 3*y - 4*x + 1

f1_der_x = lambda x, y: 2*x - 3*y + 11
f1_der_y = lambda x, y: x**2 + x + 7
f2_der_x = lambda x, y: 3*x**2 + 3*y - 3
f2_der_y = lambda x, y: x**3 - 4*x + 4

X0 = [-4, 0]
Y0 = [4, 7]

f = lambda x, y: [[f1(x,y)], [f2(x,y)]]
df = lambda x, y: [[f1_der_x(x,y), f1_der_y(x,y)], [f2_der_x(x,y), f2_der_y(x,y)]]
intervals = [[X0], [Y0]]

kravchyk_system(f, df, intervals)

1	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
2	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
3	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
4	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
5	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
6	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
7	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
8	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
9	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
10	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
11	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
12	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
13	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
14	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
15	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 7.0000000]	3.0
16	*	X	[-4.0000000, 0.0000000]	4.0	Y	[4.0000000, 

ComponentError: ignored